# Installing Dependencies

In [135]:
import sys

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing matplotlib
!{sys.executable} -m pip install matplotlib

# Installing statsmodels
!{sys.executable} -m pip install statsmodels

# Installing tqdm
!{sys.executable} -m pip install tqdm

# Installing requests
!{sys.executable} -m pip install requests

# Installing beautiful soup
!{sys.executable} -m pip install beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [136]:
# Import Statistic Libraries
import re
import requests

import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

# Import Utility Libraries
from typing                     import List, Dict
from tqdm                       import tqdm
from matplotlib.lines           import Line2D
from bs4                        import BeautifulSoup

# Utility functions

In [148]:
def get_page(number_of_page: int ) -> pd.DataFrame:
    page_url : str = f"https://kamernet.nl/en/for-rent/rooms-netherlands?radius=5&minSize=&maxRent=&searchview=1&typeAndCity=rooms-netherlands&pageNo={number_of_page}"

    page_response = requests.get(url= page_url)
    soup = BeautifulSoup(page_response.content, 'html.parser')
    listings = soup.find_all("a", { "class" : "MuiCard-root"})
    listing_dictionary : Dict[str : list] = {
        "street" : [],
        "city" : [],
        "size" : [],
        "furnishing" : [],
        "type" : [],
        "date" : [],
        "utilities": [],
        "price": [],
        "url": []
    }

    for listing in listings:
        locations = listing.findChildren("h6" , recursive=True)
        data_utilities = listing.findChildren("p", recursive=True)
        price = listing.findChild("h5", recursive = True)


        listing_street = locations[0].text
        listing_city = locations[1].text

        listing_size = 0
        listing_furnishing = ""
        listing_type = ""
        listing_date = ""
        listing_utilities = ""
        listing_price = 0

        if( len(data_utilities) == 5):
            listing_size = data_utilities[0].text
            listing_furnishing = data_utilities[1].text
            listing_type = data_utilities[2].text.replace(",", " ")
            listing_date = data_utilities[3].text
            listing_utilities = data_utilities[4].text
            listing_price = price.text

        if( len(data_utilities) ==  6):
            listing_size = data_utilities[0].text
            listing_furnishing = data_utilities[1].text
            listing_type = data_utilities[2].text.replace(",", " ") + data_utilities[3].text
            listing_date = data_utilities[4].text
            listing_utilities = data_utilities[5].text
            listing_price = price.text

        listing_dictionary['street'].append(listing_street)
        listing_dictionary['city'].append(listing_city)
        listing_dictionary['size'].append(listing_size)
        listing_dictionary['furnishing'].append(listing_furnishing)
        listing_dictionary['type'].append(listing_type)
        listing_dictionary['date'].append(listing_date)
        listing_dictionary['utilities'].append(listing_utilities[1:])
        listing_dictionary['price'].append(listing_price[1:])
        listing_dictionary['url'].append(f'https://kamernet.nl{listing.get('href')}')

    return pd.DataFrame(listing_dictionary)


# Scrapring


In [149]:
page_list : List[pd.DataFrame] = []

for i in tqdm(range(2, 42), desc="Processing"):

    page_list.append(get_page(i))

combined_df = pd.concat(page_list, ignore_index=True)

output_file = "../data/combined_data.csv"
combined_df.to_csv(output_file, index=False, sep=';')

Processing: 100%|██████████| 40/40 [01:25<00:00,  2.13s/it]


In [150]:
combined_df

,street,city,size,furnishing,type,date,utilities,price,url
0,"Galjoenstraat,",Zaandam,16 m²,furnished,Roomin a studenthouse,From 3 Mar 2024,month,800,https://kamernet.nl/en/for-rent/room-zaandam/g...
1,"Stoofsteeg,",Gouda,58 m²,unfurnished,Apartment,From 1 Apr 2024,month,"1,325",https://kamernet.nl/en/for-rent/apartment-goud...
2,"Beeklaan,",Den Haag,17 m²,furnished,Studio,3 Mar 2024 - 1 Mar 2026,month,725,https://kamernet.nl/en/for-rent/studio-den-haa...
3,"Fuutlaan,",Eindhoven,16 m²,unfurnished,Room,From 16 Feb 2024,month incl. utilities,295,https://kamernet.nl/en/for-rent/room-eindhoven...
4,"Meidoornsingel,",Rotterdam,20 m²,furnished,Room,1 Sep 2024 - 1 Mar 2025,month incl. utilities,925,https://kamernet.nl/en/for-rent/room-rotterdam...
...,...,...,...,...,...,...,...,...,...
1195,"Pleinweg,",Rotterdam,85 m²,furnished,Apartment,From 1 Jul 2024,month,"1,300",https://kamernet.nl/en/for-rent/apartment-rott...
1196,"Graaf Balderikstraat,",Rotterdam,28 m²,unfurnished,Room,From 1 Apr 2024,month incl. utilities,"1,000",https://kamernet.nl/en/for-rent/room-rotterdam...
1197,"Adelbert van Scharnlaan H,",Maastricht,10 m²,furnished,Room,From 15 Mar 2024,month incl. utilities,300,https://kamernet.nl/en/for-rent/room-maastrich...
1198,"Koemarkt,",Purmerend,8 m²,furnished,Room,From 16 Feb 2024,month incl. utilities,600,https://kamernet.nl/en/for-rent/room-purmerend...
